# Load Dataset

In [ ]:
# Membaca dan impor dataset
import pandas as pd
df = pd.read_csv(r'D:\Praktikum_IPSD\Tubes_AI\Bengkel.csv', delimiter=';')

In [ ]:
# Menampilkan beberapa data awal 
df.head()

,ID_Bengkel,Kualitas_Service,Harga
0,OTO001,7,2
1,OTO002,1,2
2,OTO003,10,5
3,OTO004,3,3
4,OTO005,5,6
...,...,...,...
95,OTO096,10,10
96,OTO097,9,7
97,OTO098,2,9
98,OTO099,7,2


# Fuzzification

In [ ]:
# Definisi fungsi keanggotaan fuzzy nilai rendah, sedang, dan tinggi
def fungsi_turun(x, a, b):
    if x <= a:
        return 1
    elif a < x < b:
        return (b - x) / (b - a)
    else:
        return 0

def fungsi_naik(x, a, b):
    if x <= a:
        return 0
    elif a < x < b:
        return (x - a) / (b - a)
    else:
        return 1

def fungsi_segitiga(x, a, b, c):
    if x <= a or x >= c:
        return 0
    elif a < x <= b:
        return (x - a) / (b - a)
    elif b < x < c:
        return (c - x) / (c - b)

In [ ]:
# Fuzzifikasi pada nilai kualitas servis
def fuzzifikasi_servis(x):
    buruk = fungsi_turun(x, 2, 4)
    cukup = fungsi_segitiga(x, 3, 6, 9)
    bagus = fungsi_naik(x, 8, 10)
    return buruk, cukup, bagus

# Fuzzifikasi pada nilai kualitas harga
def fuzzifikasi_harga(x):
    murah = fungsi_turun(x, 2, 4)
    sedang = fungsi_segitiga(x, 3, 6, 9)
    mahal = fungsi_naik(x, 8, 10)
    return murah, sedang, mahal

# Inferensi

In [ ]:
Skor_tolak = 20
Skor_pertimbangan = 60
Skor_terima = 100

# Aturan fuzzy Sugeno dengan kombinasi kualitas dan harga
def inferensi_sugeno(s_buruk, s_cukup, s_bagus, h_murah, h_sedang, h_mahal):
    rules = []
    a1 = min(s_buruk, h_murah)
    rules.append((a1, Skor_tolak))
    
    a2 = min(s_buruk, h_sedang)
    rules.append((a2, Skor_tolak))

    a3 = min(s_buruk, h_mahal)
    rules.append((a3, Skor_tolak))
    
    a4 = min(s_cukup, h_murah)
    rules.append((a4, Skor_terima))
    
    a5 = min(s_cukup, h_sedang)
    rules.append((a5, Skor_pertimbangan))
    
    a6 = min(s_cukup, h_mahal)
    rules.append((a6, Skor_tolak))
    
    a7 = min(s_bagus, h_murah)
    rules.append((a7, Skor_terima))
    
    a8 = min(s_bagus, h_sedang)
    rules.append((a8, Skor_terima))
    
    a9 = min(s_bagus, h_mahal)
    rules.append((a9, Skor_pertimbangan))
    
    return rules

# Defuzzification

In [ ]:
# Menghitung nilai akhir menggunakan metode defuzzifikasi Sugeno 
def defuzzifikasi_averange(rules):
    total_alpha_z = 0.0
    total_alpha = 0.0
    
    for alpha, z in rules:
        total_alpha_z += alpha * z
        total_alpha += alpha
        
    if total_alpha == 0:
        return 0.0
    
    return total_alpha_z / total_alpha

In [ ]:
hasil = []
# Memproses seluruh data bengkel untuk mendapatkan skor akhir keputusan
for index, row in df.iterrows():
    id_bengkel = row['ID_Bengkel']
    servis = row['Kualitas_Service']
    harga = row['Harga']
    
    s_vals = fuzzifikasi_servis(servis) 
    h_vals = fuzzifikasi_harga(harga)   

    rules = inferensi_sugeno(s_vals[0], s_vals[1], s_vals[2], 
                            h_vals[0], h_vals[1], h_vals[2])
    
    skor_sugeno = defuzzifikasi_averange(rules)
    hasil.append([id_bengkel, servis, harga, skor_sugeno])

In [ ]:
# Menyusun dan mengurutkan hasil akhir berdasarkan skor tertinggi
df_hasil = pd.DataFrame(hasil, columns=['ID', 'Servis', 'Harga', 'Skor'])
df_sorted = df_hasil.sort_values(by='Skor', ascending=False)
# Menampilkan hasil akhir perhitungan dalam bentuk tabel
df_sorted

,ID,Servis,Harga,Skor
20,OTO021,10,2,100.0
23,OTO024,9,2,100.0
13,OTO014,9,6,100.0
8,OTO009,9,7,100.0
32,OTO033,7,3,100.0
...,...,...,...,...
84,OTO085,7,10,20.0
85,OTO086,3,1,20.0
94,OTO095,5,10,20.0
97,OTO098,2,9,20.0
